# MCQ Generator with LangChain and Gemini

This notebook demonstrates how to build a Multiple Choice Question (MCQ) generator using LangChain and Google's Gemini API. The goal of this project is to automatically generate a quiz from a given text, and then evaluate the generated quiz for quality and relevance.

This project is a great way to showcase skills in:

*   **Natural Language Processing (NLP):** Using large language models (LLMs) to understand and process text.
*   **LangChain:** Building complex applications with LLMs by chaining together different components.
*   **API Integration:** Interacting with the Google Generative AI API to leverage powerful models like Gemini.
*   **Prompt Engineering:** Designing effective prompts to guide the LLM's output.
*   **Python and Jupyter Notebooks:** Writing clean, well-documented code to solve a real-world problem.

In [36]:
!pip install langchain-google-genai

## 1. Import Necessary Libraries

In [37]:
import os
import json
import pandas as pd
import traceback
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_google_genai import ChatGoogleGenerativeAI
import PyPDF2

## 2. Environment Configuration

Next, we'll load the Google API key from a `.env` file. This is a good practice for managing sensitive information like API keys.

In [40]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

In [41]:
# Replace with your Google API Key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

## 3. LangChain Setup

Now we'll set up the core components of our LangChain application.

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7)

### 3.1. Define Prompt Templates

We need two prompt templates: one for generating the MCQs and another for evaluating them.

In [56]:
TEMPLATE = '''
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to 
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. 
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
'''

In [57]:
TEMPLATE2 = '''
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
'''

### 3.2. Create LangChain Chains

We'll create two `LLMChain` instances: one for quiz generation and one for evaluation. Then, we'll combine them into a `SequentialChain`.

In [58]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [59]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [60]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [61]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [62]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

## 4. Data Preparation

Now we'll load the text from the `data.txt` file. This text will be used as the source material for generating the quiz.

In [63]:
file_path = "../data.txt"

In [64]:
with open(file_path, 'r') as file:
    TEXT = file.read()

## 5. MCQ Generation and Evaluation

Now it's time to run our chain and generate the MCQs. We'll specify the number of questions, the subject, and the tone of the quiz.

In [65]:
NUMBER = 5
SUBJECT = "biology"
TONE = "simple"
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [66]:
# Run the chain to generate and evaluate the quiz
response = generate_evaluate_chain.invoke(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)

### 5.1. Display the Results

In [70]:
print("--- RAW MODEL OUTPUT FOR 'quiz' ---")
print(repr(response['quiz']))
print("--- END OF RAW OUTPUT ---")

## 6. Output Processing

Finally, we'll process the output from the LangChain pipeline. We'll parse the generated quiz, format it as a Pandas DataFrame, and save it to a CSV file.

In [ ]:
import json
import pandas as pd

# --- 1. Get the raw string output from the model ---
raw_quiz_output = response.get("quiz")
print("--- RAW MODEL OUTPUT ---")
print(raw_quiz_output)


# --- 2. Clean the string to extract only the JSON part ---
# We find the first '{' and the last '}' to isolate the JSON object
try:
    start_index = raw_quiz_output.find('{')
    end_index = raw_quiz_output.rfind('}') + 1
    cleaned_json_string = raw_quiz_output[start_index:end_index]
    
    print("\n--- CLEANED JSON STRING ---")
    print(cleaned_json_string)
    
    # --- 3. Parse the cleaned string ---
    quiz_data = json.loads(cleaned_json_string)
    print("\n--- SUCCESSFULLY PARSED! ---")

    # --- 4. Continue with the rest of your logic ---
    quiz_table_data = []
    for key, value in quiz_data.items():
        mcq = value["mcq"]
        options = " | ".join(
            [
                f"{option}: {option_value}"
                for option, option_value in value["options"].items()
            ]
        )
        correct = value["correct"]
        quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

    quiz_df = pd.DataFrame(quiz_table_data)
    
    # Display the final DataFrame
    print("\n--- FINAL QUIZ DATAFRAME ---")
    display(quiz_df)

    # Save to CSV
    quiz_df.to_csv("biology_quiz.csv", index=False)
    print("\nSuccessfully saved to biology_quiz.csv")


except (AttributeError, IndexError, json.JSONDecodeError) as e:
    print("\n--- FAILED TO CLEAN OR PARSE THE STRING ---")
    print("There was an error processing the model's output. It might be in an unexpected format.")
    print(f"Error: {e}")